In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import fitz
import re
from PyPDF2 import PdfReader
# import PyPDF2
# import jpype
# import pdfplumber
# from openpyxl import Workbook

if os.path.abspath('../') not in sys.path:
    sys.path.insert(0, os.path.abspath('../'))

In [2]:
econ_input = '../data/socio-economic-data-of-63-provinces-and-centrally-run-cities-2015-2021.pdf'
econ_output = '../data/Vietcon-extracted-feature'
econ_output_2 = '../data/Vietcon-extracted-feature'
output_csv = '../data/Viecon_data.csv'
features = [
    "AREA OF LAND (Thous. ha)",
    "POPULATION DENSITY (Person/km2)",
    "At current prices (Bill. dongs)",
    "State budget revenue (Bill. dongs)",
    "State budget expenditure (Bill. dongs)",
    "Investment at current prices (Bill. dongs)",
    "Area of floors of residential buildings constructed in the year by types of house (Thous. m2)",
    "Number of newly established enterprises (Enterprise)",
    "Số trang trại - Number of farms",
    "Planted area of cereals (Thous. ha)",
    "Poultry population (Mill. heads)",
    "Production of fishery (Ton)",
    "Index of industrial production (%)", 
    "Retail sales of goods at current prices (Bill. dongs)",
    "Turnover of travelling at current prices (Bill. dongs)",
    "Number of passengers carried of local tranport (Mill. persons)"
]
provinces = [
    'An Giang', 'Ba Ria-Vung Tau', 'Bac Lieu', 'Bac Kan', 'Bac Giang',
    'Bac Ninh', 'Ben Tre', 'Binh Duong', 'Binh Dinh', 'Binh Phuoc',
    'Binh Thuan', 'Ca Mau', 'Can Tho', 'Cao Bang', 'Da Nang', 'Dak Lak',
    'Dak Nong', 'Dien Bien', 'Dong Nai', 'Dong Thap', 'Gia Lai', 'Ha Giang',
    'Ha Nam', 'Ha Noi', 'Ha Tinh', 'Hai Duong', 'Hai Phong', 'Hau Giang',
    'Hoa Binh', 'Hung Yen', 'Khanh Hoa', 'Kien Giang', 'Kon Tum', 'Lai Chau',
    'Lam Dong', 'Lang Son', 'Lao Cai', 'Long An', 'Nam Dinh', 'Nghe An',
    'Ninh Binh', 'Ninh Thuan', 'Phu Tho', 'Phu Yen', 'Quang Binh', 'Quang Nam',
    'Quang Ngai', 'Quang Ninh', 'Quang Tri', 'Soc Trang', 'Son La', 'Tay Ninh',
    'Thai Binh', 'Thai Nguyen', 'Thanh Hoa', 'Thua Thien - Hue', 'Tien Giang',
    'Tra Vinh', 'Tuyen Quang', 'Vinh Long', 'Vinh Phuc', 'Yen Bai'
]


In [3]:
def extract_data_from_pdf(pdf_path, features_list, provinces_list):
    pdf_document = fitz.open(pdf_path)

    # Initialize a dictionary to store extracted data
    data_dict = {province: [] for province in provinces_list}

    # Iterate through each page in the PDF
    for page_num in range(pdf_document.page_count):
        page = pdf_document[page_num]
        
        page_text = page.get_text()

        # Find occurrences of provinces on the page
        occurrences = [(province, match.start()) for province in provinces_list for match in re.finditer(province, page_text)]

        # Iterate through each occurrence
        for province, start_index in occurrences:
            province_text = page_text[start_index:]
            next_province_index = min((occ[1] for occ in occurrences if occ[1] > start_index), default=len(page_text))
            province_text = province_text[:next_province_index]

            # Use regular expressions to extract data for all specified features
            features_data = {}
            for feature in features_list:
                # Construct the regular expression pattern based on the feature
                pattern = re.escape(feature) + r"\s+([\d.,\s]+)"

                # Find matches
                matches = re.findall(pattern, province_text)
                if matches:
                    features_data[feature] = list(map(float, matches[0].replace(',', '.').split()))

            # Append the extracted data to the corresponding province in the dictionary
            data_dict[province].append(features_data)

    # Close the PDF file
    pdf_document.close()

    return data_dict

# Extract data from PDF
result = extract_data_from_pdf('../data/socio-economic-data-of-63-provinces-and-centrally-run-cities-2015-2021.pdf', features, provinces)
# Display the result
for province, occurrences_data in result.items():
    for index, features_data in enumerate(occurrences_data, start=1):
        if any(features_data.values()):
            print(f"{province} : {features_data}")


An Giang : {'AREA OF LAND (Thous. ha)': [353.7, 353.7, 353.6, 353.7, 353.7, 353.7, 353.7]}
An Giang : {'POPULATION DENSITY (Person/km2)': [566.0, 559.0, 553.0, 546.0, 539.0, 538.0, 540.0]}
An Giang : {'At current prices (Bill. dongs)': [60466.8, 65466.9, 70719.7, 78152.9, 84939.6, 88959.2, 92237.9]}
An Giang : {'State budget revenue (Bill. dongs)': [13141.9, 14282.0, 16165.1, 19303.5, 20539.3, 21865.4, 21757.1], 'State budget expenditure (Bill. dongs)': [17687.6, 18002.0, 22522.4, 24880.1, 26701.5, 29402.7, 26782.3]}
An Giang : {'Investment at current prices (Bill. dongs)': [11228.7, 11920.3, 13488.6, 16146.9, 17695.7, 19804.4, 13186.2]}
An Giang : {'Number of newly established enterprises (Enterprise)': [619.0, 778.0, 724.0, 720.0, 858.0, 604.0, 57.0]}
An Giang : {'Planted area of cereals (Thous. ha)': [652.8, 677.1, 649.4, 630.6, 632.2, 642.9, 630.2]}
An Giang : {'Production of fishery (Ton)': [348079.0, 369843.0, 401724.0, 486804.0, 539909.0, 511148.0, 522253.0], 'Index of industria

In [4]:
def split_features_data(features_data):
    features = list(features_data.keys())
    data = list(features_data.values())

    if isinstance(features[0], str):
        return [features], [data]

    # Nếu có nhiều feature lists, tách chúng thành từng cặp features và data riêng biệt
    if isinstance(features[0], list):
        features_list = []
        data_list = []
        for i in range(len(features)):
            features_list.append(features[i])
            data_list.append(data[i])
        return features_list, data_list
    return features, data

# Dùng trong vòng lặp
for province, occurrences_data in result.items():
    for index, features_data in enumerate(occurrences_data, start=1):
        if any(features_data.values()):
            features_list, data_list = split_features_data(features_data)
            for features, data in zip(features_list, data_list):
                print(f"Province:{province}")
                print(f"Features: {features}")
                print(f"Data: {data}")
                print()


Province:An Giang
Features: ['AREA OF LAND (Thous. ha)']
Data: [[353.7, 353.7, 353.6, 353.7, 353.7, 353.7, 353.7]]

Province:An Giang
Features: ['POPULATION DENSITY (Person/km2)']
Data: [[566.0, 559.0, 553.0, 546.0, 539.0, 538.0, 540.0]]

Province:An Giang
Features: ['At current prices (Bill. dongs)']
Data: [[60466.8, 65466.9, 70719.7, 78152.9, 84939.6, 88959.2, 92237.9]]

Province:An Giang
Features: ['State budget revenue (Bill. dongs)', 'State budget expenditure (Bill. dongs)']
Data: [[13141.9, 14282.0, 16165.1, 19303.5, 20539.3, 21865.4, 21757.1], [17687.6, 18002.0, 22522.4, 24880.1, 26701.5, 29402.7, 26782.3]]

Province:An Giang
Features: ['Investment at current prices (Bill. dongs)']
Data: [[11228.7, 11920.3, 13488.6, 16146.9, 17695.7, 19804.4, 13186.2]]

Province:An Giang
Features: ['Number of newly established enterprises (Enterprise)']
Data: [[619.0, 778.0, 724.0, 720.0, 858.0, 604.0, 57.0]]

Province:An Giang
Features: ['Planted area of cereals (Thous. ha)']
Data: [[652.8, 677

In [5]:
extracted_data_list = []

# Iterate through the result
for province, occurrences_data in result.items():
    for index, features_data in enumerate(occurrences_data, start=1):
        if any(features_data.values()):
            features_list, data_list = split_features_data(features_data)
            
            for features, data in zip(features_list, data_list):
                # Initialize a dictionary to store the current occurrence's data
                occurrence_data = {
                    'Province': province,
                    'Features': [],
                    'Data': []
                }
                
                if len(features) == 1:
                    # If there is only one feature, add it to the dictionary
                    feature_name = features[0]
                    occurrence_data['Features'].append(feature_name)
                    occurrence_data['Data'].append(data[0])
                else:
                    # If there are multiple features, add each one to the dictionary
                    for feat, dat in zip(features, data):
                        feature_name = f"{province}_{feat}"  # Combine province name with feature
                        occurrence_data['Features'].append(feature_name)
                        occurrence_data['Data'].append(dat)
                
                # Append the current occurrence's data to the list
                extracted_data_list.append(occurrence_data)
print('\n',extracted_data_list)



 [{'Province': 'An Giang', 'Features': ['AREA OF LAND (Thous. ha)'], 'Data': [[353.7, 353.7, 353.6, 353.7, 353.7, 353.7, 353.7]]}, {'Province': 'An Giang', 'Features': ['POPULATION DENSITY (Person/km2)'], 'Data': [[566.0, 559.0, 553.0, 546.0, 539.0, 538.0, 540.0]]}, {'Province': 'An Giang', 'Features': ['At current prices (Bill. dongs)'], 'Data': [[60466.8, 65466.9, 70719.7, 78152.9, 84939.6, 88959.2, 92237.9]]}, {'Province': 'An Giang', 'Features': ['An Giang_State budget revenue (Bill. dongs)', 'An Giang_State budget expenditure (Bill. dongs)'], 'Data': [[13141.9, 14282.0, 16165.1, 19303.5, 20539.3, 21865.4, 21757.1], [17687.6, 18002.0, 22522.4, 24880.1, 26701.5, 29402.7, 26782.3]]}, {'Province': 'An Giang', 'Features': ['Investment at current prices (Bill. dongs)'], 'Data': [[11228.7, 11920.3, 13488.6, 16146.9, 17695.7, 19804.4, 13186.2]]}, {'Province': 'An Giang', 'Features': ['Number of newly established enterprises (Enterprise)'], 'Data': [[619.0, 778.0, 724.0, 720.0, 858.0, 604

In [6]:
#pivot the extracted data list
pivot_list = []

for occurrence_data in extracted_data_list:
    # Get the province name
    province = occurrence_data['Province']
    
    # Get the features and data
    features = occurrence_data['Features']
    data = occurrence_data['Data']
    
    # Create a dictionary to store the current occurrence's data
    pivot_data = {
        'Province': province
    }
    
    # Add the features and data to the dictionary
    for feature, dat in zip(features, data):
        pivot_data[feature] = dat
    
    # Append the current occurrence's data to the list
    pivot_list.append(pivot_data)
print('\n',pivot_list)


 [{'Province': 'An Giang', 'AREA OF LAND (Thous. ha)': [353.7, 353.7, 353.6, 353.7, 353.7, 353.7, 353.7]}, {'Province': 'An Giang', 'POPULATION DENSITY (Person/km2)': [566.0, 559.0, 553.0, 546.0, 539.0, 538.0, 540.0]}, {'Province': 'An Giang', 'At current prices (Bill. dongs)': [60466.8, 65466.9, 70719.7, 78152.9, 84939.6, 88959.2, 92237.9]}, {'Province': 'An Giang', 'An Giang_State budget revenue (Bill. dongs)': [13141.9, 14282.0, 16165.1, 19303.5, 20539.3, 21865.4, 21757.1], 'An Giang_State budget expenditure (Bill. dongs)': [17687.6, 18002.0, 22522.4, 24880.1, 26701.5, 29402.7, 26782.3]}, {'Province': 'An Giang', 'Investment at current prices (Bill. dongs)': [11228.7, 11920.3, 13488.6, 16146.9, 17695.7, 19804.4, 13186.2]}, {'Province': 'An Giang', 'Number of newly established enterprises (Enterprise)': [619.0, 778.0, 724.0, 720.0, 858.0, 604.0, 57.0]}, {'Province': 'An Giang', 'Planted area of cereals (Thous. ha)': [652.8, 677.1, 649.4, 630.6, 632.2, 642.9, 630.2]}, {'Province': 'A

In [25]:
# create a data based on the pivot list 
Vieconomic_data = pd.DataFrame(pivot_list)
Vieconomic_data.head(20)

,Province,AREA OF LAND (Thous. ha),POPULATION DENSITY (Person/km2),At current prices (Bill. dongs),An Giang_State budget revenue (Bill. dongs),An Giang_State budget expenditure (Bill. dongs),Investment at current prices (Bill. dongs),Number of newly established enterprises (Enterprise),Planted area of cereals (Thous. ha),An Giang_Production of fishery (Ton),...,Vinh Phuc_Production of fishery (Ton),Vinh Phuc_Index of industrial production (%),Yen Bai_State budget revenue (Bill. dongs),Yen Bai_State budget expenditure (Bill. dongs),Yen Bai_Số trang trại - Number of farms,Yen Bai_Planted area of cereals (Thous. ha),Yen Bai_Production of fishery (Ton),Yen Bai_Index of industrial production (%),Yen Bai_Retail sales of goods at current prices (Bill. dongs),Yen Bai_Turnover of travelling at current prices (Bill. dongs)
0,An Giang,"[353.7, 353.7, 353.6, 353.7, 353.7, 353.7, 353.7]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,An Giang,NaN,"[566.0, 559.0, 553.0, 546.0, 539.0, 538.0, 540.0]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,An Giang,NaN,NaN,"[60466.8, 65466.9, 70719.7, 78152.9, 84939.6, ...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,An Giang,NaN,NaN,NaN,"[13141.9, 14282.0, 16165.1, 19303.5, 20539.3, ...","[17687.6, 18002.0, 22522.4, 24880.1, 26701.5, ...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,An Giang,NaN,NaN,NaN,NaN,NaN,"[11228.7, 11920.3, 13488.6, 16146.9, 17695.7, ...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,An Giang,NaN,NaN,NaN,NaN,NaN,NaN,"[619.0, 778.0, 724.0, 720.0, 858.0, 604.0, 57.0]",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,An Giang,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[652.8, 677.1, 649.4, 630.6, 632.2, 642.9, 630.2]",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,An Giang,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[348079.0, 369843.0, 401724.0, 486804.0, 53990...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,An Giang,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Ba Ria-Vung Tau,"[198.1, 198.1, 197.9, 198.1, 198.1, 198.3, 198.3]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
for column in Vieconomic_data.columns:
    if column != 'Province':  # Skip the 'Province' column
        Vieconomic_data[column] = Vieconomic_data[column].apply(lambda x: pd.Series(x) if isinstance(x, list) else x)

# Step 2: Combine the resulting dataframes for each province
result_dfs = []

for province in Vieconomic_data['Province'].unique():
    province_df = Vieconomic_data[Vieconomic_data['Province'] == province]
    result_df = pd.concat([province_df['Province']] + [province_df[column] for column in Vieconomic_data.columns if column != 'Province'], axis=1)
    result_dfs.append(result_df)

# Step 3: Concatenate the dataframes for all provinces
final_result_df = pd.concat(result_dfs, ignore_index=True)

# Step 4: Reset index
final_result_df.reset_index(drop=True, inplace=True)
final_result_df.head(20)

,Province,AREA OF LAND (Thous. ha),POPULATION DENSITY (Person/km2),At current prices (Bill. dongs),An Giang_State budget revenue (Bill. dongs),An Giang_State budget expenditure (Bill. dongs),Investment at current prices (Bill. dongs),Number of newly established enterprises (Enterprise),Planted area of cereals (Thous. ha),An Giang_Production of fishery (Ton),...,Vinh Phuc_Production of fishery (Ton),Vinh Phuc_Index of industrial production (%),Yen Bai_State budget revenue (Bill. dongs),Yen Bai_State budget expenditure (Bill. dongs),Yen Bai_Số trang trại - Number of farms,Yen Bai_Planted area of cereals (Thous. ha),Yen Bai_Production of fishery (Ton),Yen Bai_Index of industrial production (%),Yen Bai_Retail sales of goods at current prices (Bill. dongs),Yen Bai_Turnover of travelling at current prices (Bill. dongs)
0,An Giang,353.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,An Giang,353.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,An Giang,353.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,An Giang,353.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,An Giang,353.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,An Giang,353.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,An Giang,353.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,An Giang,NaN,0 566.0 1 559.0 2 553.0 3 546.0 4 ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,An Giang,NaN,NaN,0 60466.8 1 65466.9 2 70719.7 3 78...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,An Giang,NaN,NaN,NaN,0 13141.9 1 14282.0 2 16165.1 3 19...,0 17687.6 1 18002.0 2 22522.4 3 24...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_year_lists = Vieconomic_data[['Year', 'AREA OF LAND (Thous. ha)']]

# Step 2: Explode the lists while keeping the 'Year' values
df_exploded = df_year_lists.explode('AREA OF LAND (Thous. ha)')

# Step 3: Reset index
df_exploded.reset_index(drop=True, inplace=True)

In [9]:
# def print_entry_data(entry):
#     print(f"Province: {entry['Province']}")
#     for feature, data in zip(entry['Features'], entry['Data']):
#         print(f"  Feature: {feature}")
#         print(f"  Data: {', '.join(map(str, data))}")
#     print()

# for entry in extracted_data_list:
#     print_entry_data(entry)


In [10]:
# # open 
# econ = pd.read_excel('../data/Viecon_data.xlsx')

In [11]:
# # rename the Unnamed: 0 column to Provinces
# econ.rename(columns={'Unnamed: 0':'Provinces'}, inplace=True)
# econ.head(5)

In [12]:
# #multiply every province by 7 times 
# econ = pd.concat([econ]*7, ignore_index=True)
# # sort the provinces in alphabetical order
# econ = econ.sort_values(by=['Provinces'])
# # add a year column with values from 2015 to 2021 
# years = np.tile([2015, 2016, 2017, 2018, 2019, 2020, 2021], len(provinces))
# econ['Year'] = years

In [13]:
# def extract_feature_values(text, feature_dir):
#     """Extracts feature values from the PDF text."""
#     # Implement your feature extraction logic here, tailored to your PDF structure
#     # This might involve regular expressions, table parsing, or other techniques
#     # Example (replace with your actual implementation):
#     feature_values = {
#         "Index of industrial production (%)": ...,
#         "Production of wood (Thous. m3)": ...,
#         # ... (extract values for other features)
#     }
#     return feature_values

# # Specify data directory
# data_dir = "clean/data/Viecon-extracted-feature"

# # Create DataFrame with provinces as index and features as columns
# data_df = pd.DataFrame(index=provinces, columns=["Year"] + features)

# # Iterate through feature directories, provinces, and PDF files
# for feature_dir in os.listdir(data_dir):
#     feature_path = os.path.join(data_dir, feature_dir)
#     for province_dir in os.listdir(feature_path):
#         province_path = os.path.join(feature_path, province_dir)
#         for filename in os.listdir(province_path):
#             if filename.endswith(".pdf"):
#                 file_path = os.path.join(province_path, filename)

#                 # Extract text from PDF
#                 with open(file_path, 'rb') as pdf_file:
#                     pdf_reader = PdfReader(pdf_file)
#                     page = pdf_reader.pages[0]  # Adjust page number if needed
#                     text = page.extract_text()

#                 # Extract year and feature values
#                 year = extract_year_from_filename(filename)
#                 feature_values = extract_feature_values(text, feature_dir)

#                 # Fill in the DataFrame
#                 data_df.loc[province_dir, "Year"] = year
#                 data_df.loc[province_dir, features] = feature_values

# # Save the extracted data as an Excel file
# data_df.to_excel("extracted_data.xlsx", index=True)
